<a href="https://colab.research.google.com/github/unknown-jun/Deep_learning/blob/main/%EB%8F%99%EB%AC%BC%EB%B6%84%EB%A5%98%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-771b0a94-4262-8b78-b289-b6a51d146afd)


In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%tensorflow_version 1.x # 코랩에서 텐서플로우 1.x 버전으로 코드를 수행하기 위한 명령어

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # 코랩에서 텐서플로우 1.x 버전으로 코드를 수행하기 위한 명령어`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [4]:
"""지금까지 제일 잘 분류하는 신경망 코드 """
%cd /content/gdrive/MyDrive

#import tensorflow as tf
from keras.models import *
from keras.layers import  Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation
from sklearn.model_selection import train_test_split
from keras.datasets.fashion_mnist import load_data
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
import loader3
from PIL import Image

/content/gdrive/MyDrive


In [5]:
"""이미지 경로 설정 """

train_image = '/content/gdrive/MyDrive/samples6/train_resize2/train_resize3'
test_image = '/content/gdrive/MyDrive/samples6/test_resize2/test_resize3'
train_label = '/content/gdrive/MyDrive/samples6/train_label.csv'
test_label = '/content/gdrive/MyDrive/samples6/test_label.csv'



""" loader3 함수를 가상환경 밑에 넣고 함수 구현 """

x_train = loader3.image_load(train_image)  # 훈련 이미지를 불어와서 숫자로 변환
y_train = loader3.label_load(train_label)  # 레이블을 불러와서 one hot encoding한다

x_test = loader3.image_load(test_image)  # 테스트 이미지를 불러와서 숫자로 변환
y_test = loader3.label_load(test_label)  # 레이블을 불러와서 one hot encoding한다       

  
print ( loader3.image_load(train_image).shape )  # (1600, 32, 32, 3)
print ( loader3.image_load(test_image).shape )   # (400,32,32,3)
print ( loader3.label_load(train_label).shape)   # (1600, 2)
print ( loader3.label_load(test_label).shape )   # (400,2)

"""4차원 데이터이므로 reshape 필요없음"""

x_train = x_train.astype('float')  # 훈련 데이터의 숫자를 실수형으로 변경
x_test = x_test.astype('float')    # 테스트 데이터의 숫자를 실수형으로 변경
x_train = x_train / 255
x_test = x_test /255

(1600, 32, 32, 3)
(400, 32, 32, 3)
(1600, 2)
(400, 2)


In [6]:
# 검증 데이터셋을 만듭니다.

from sklearn.model_selection import train_test_split
np.random.seed(111)
# 훈련/테스트 데이터를 0.8/0.3의 비율로 분리합니다.

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size = 0.2, random_state = 777)

print(x_train.shape)
print(y_val.shape)

(1280, 32, 32, 3)
(320, 2)


In [7]:
## 제너레이터 사용 (이미지를 증식시키는 코드)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip = True,
                                vertical_flip=True,
                                shear_range=0.5,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   rotation_range = 30,
                                   fill_mode = 'nearest')

val_datagen = ImageDataGenerator()

batch_size = 32 # 사진 한장 당 batch_size만큼 증식시킴

train_generator = train_datagen.flow(x_train, y_train,
                                    batch_size =32)

val_generator = val_datagen.flow(x_val, y_val,
                                batch_size =32)


#텐서플로우 2.x의 전이학습 기능
- 전이 학습은 사전에 학습된 네트워크의 가중치 또는 이미지넷 대회에서 우승한 유명한 신경망 모델을 가져와서 사용하는 기능을 말함함

1. 모델을 변형하지 않고 그대로 사용하는 방법
~~~
from keras.applications import VGG16
vgg16 = VGG16(weights = 'imagenet', input_shape= (32, 32, 3), include_top = False)
~~~

weights: 
- imagenet데이터를 학습시킨 가중치의 사용여부를 결정한다. 
- 기본값은 None(1000개의 이미지 사진들이 있고 그 사진들을 학습시킨 가중치)

input_shape 
- 입력데이터의 사이즈를 기술한다.

include_top
- 완전연결계층의 모델의 분류기를 내가 직접 기술할지 말지를 결정한다
- False로 되어 있으면 내가 직접 완전연결계층을 짜겠다.

In [8]:
from keras.models import *
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.applications import VGG16   # 이미지 대회에서 우승한 유명한 신경망 중에 하나인 옥스포드에서 만든 VGG16 신경망을 가져오겠다.


# imagenet을 학습한 모델을 불러옵니다.

vgg16 = VGG16(weights = 'imagenet', input_shape = (32, 32, 3), include_top = False)   


for layer in vgg16.layers[:-4]:  # cnn층의 가중치 16개 중 4개만 훈련 시키겠다.
    layer.trainable = False      # 이미지 넷 대회에서 우승한 가중치 가져다 쓰겠다.
    

model = Sequential()    

model.add(vgg16)      # cnn층까지는 옥스포드에서 설계한 vgg16모델 설계도를 사용하고 
model.add(Flatten())
model.add(Dense(256)) # 완전연결계층 1층
model.add(Activation('relu'))
model.add(Dense(128)) # 완전연결계층 2층
model.add(Activation('relu'))
model.add(Dense(64))  # 완전연결계층 3층
model.add(Activation('relu'))
model.add(Dense(2, activation = 'softmax')) # 출력층 4층

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

# 텐써플로우 2.0의 장점
1. 즉시 실행모드로 실행할 수 있다.( 파이썬처럼 실행이 가능하다 )
2. 이미지 넷 대회에서 우승한 유명한 신경망의 모델 셀계와 가중치 파일을 내가 분류하고자 하는 신경망 코드에 쉽게 가져올 수 있다.( 전이 학습 )
3. 이미지를 증식시키고 증식 시킨 이미지들을 쉽게 신경망에 입력할 수 있다.
4. EarlyStopping 기능을 구현할 수 있다.

■ EarlyStopping 기능
- EarlyStopping 콜백을 직역하면 '이른 멈춤'이다
- 즉 모델 학습시 지정된 기간동안 모니터링 하는 평가지표에서 성능향상이 일어나지 않는 경우 학습을 중단함
- 주로 많이 사용하는 콜백인자는 다음과 같다
~~~
예제:
EarlyStopping( monitor = 'val_loss', patience=2, verbose=0, mode='auto')
~~~
- monitor: 모니터링할 평가지표를 설정한다.( 오차: val_loss, 정확도: val_acc )
- verbose: 콜백의 수행과정 노충 여부를 결정한다.  
    - 0: 아무런 표시를 하지 않겠다.
    - 1: 프로그래스 바(progress bar)를 출력
    - 2: 에폭마다 수행과정을 출력
- patience: 지정한 수만큼의 기간에서 평가지표의 향상이 일어나지 않을 경우 학습을 중단하겠다. patient = 5를 썼다면 5번은 참아주겠다.
~~~
구현: 
callbacks = [ EarlyStopping( monitor = 'val_loss', patience=2, verbose=0, mode='auto') ]
model.fit(x_train, y_train, 
          batch_size = 32,
          validation_data=(x_val, y_val),
          epochs = 10,
          callbacks = callbacks )
~~~

In [11]:
# 얼룩말과 재규어를 분류하는 모델을 생성하고 가중치와 바이어스가 들어있는 모델(h5)을 저장하는 코드

from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = 're_w2.h5'

callbacks = [ ModelCheckpoint( filepath = filepath , monitor = 'val_loss' , verbose=1, save_best_only=True ) ]


model.compile(optimizer = Adam(1e-4),
             loss = 'binary_crossentropy',
             metrics = ['acc'])


batch_size=32


def get_step(train_len, batch_size):      
    if(train_len % batch_size > 0):
        return train_len // batch_size + 1
    else:
        return train_len // batch_size

"""
원래는 
history = model.fit(x_train, y_train, -> 이렇게 늘여 써야 하는데 train_generator로 한꺼번에 씀
                    epochs = 30,
                    steps_per_epoch = get_step(len(x_train), batch_size),
                    validation_data = x_val, y_val -> 이것 역시 val_generator로 축약
"""
history = model.fit(train_generator,
                    epochs = 400,
                    steps_per_epoch = get_step(len(x_train), batch_size),
                    validation_data = val_generator,
                    validation_steps = get_step(len(x_val), batch_size), callbacks=callbacks )

print( model.evaluate(x_test,y_test))

save_model(model, 're_w3.h5')

Epoch 1/400
40/40 [==============================] - 1s 22ms/step - loss: 0.0372 - acc: 0.9824 - val_loss: 0.4669 - val_acc: 0.8906

Epoch 00001: val_loss improved from inf to 0.46688, saving model to re_w2.h5
Epoch 2/400
40/40 [==============================] - 1s 18ms/step - loss: 0.0178 - acc: 0.9935 - val_loss: 0.5007 - val_acc: 0.9187

Epoch 00002: val_loss did not improve from 0.46688
Epoch 3/400
40/40 [==============================] - 1s 18ms/step - loss: 0.0152 - acc: 0.9965 - val_loss: 0.7670 - val_acc: 0.8719

Epoch 00003: val_loss did not improve from 0.46688
Epoch 4/400
40/40 [==============================] - 1s 17ms/step - loss: 0.0572 - acc: 0.9762 - val_loss: 0.4417 - val_acc: 0.9094

Epoch 00004: val_loss improved from 0.46688 to 0.44165, saving model to re_w2.h5
Epoch 5/400
40/40 [==============================] - 1s 18ms/step - loss: 0.0300 - acc: 0.9861 - val_loss: 0.5206 - val_acc: 0.9031

Epoch 00005: val_loss did not improve from 0.44165
Epoch 6/400
40/40 [=====